<a href="https://colab.research.google.com/github/monu322/CE888/blob/main/Assg2/SentimentAnalysis/Topic_number_using_HDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyLDAvis

In [3]:
import pickle
import gensim
import pyLDAvis
import pyLDAvis.gensim_models
import spacy
import pandas as pd
import nltk; nltk.download('stopwords')
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import re
import warnings
from pprint import pprint

warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#importing sentiment data
tweets=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_text.txt', header=None)
print('Training tweets', tweets.shape)

tweets_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/train_labels.txt', header=None)
print('Training labels', tweets_sentiment.shape)

tweets_test=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_text.txt', header=None)
print('Test tweets',tweets_test.shape)

tweets_test_sentiment=pd.read_fwf('https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/sentiment/test_labels.txt', header=None)
print('Test labels',tweets_test_sentiment.shape)

Training tweets (45615, 2)
Training labels (45615, 1)
Test tweets (12284, 1)
Test labels (12284, 1)


In [8]:

tweets.head()

,0,1
0,"""QT @user In the original draft of the 7th boo...",NaN
1,"""Ben Smith / Smith (concussion) remains out of...",NaN
2,Sorry bout the stream last night I crashed out...,NaN
3,Chase Headley's RBI double in the 8th inning o...,NaN
4,@user Alciato: Bee will invest 150 million in ...,NaN


In [13]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def strip_newline(series):
    return [review.replace('\n','') for review in series]

tweets[0] = strip_newline(tweets[0])
tweets[0][20:42].values

array(['@user @user 1st uneasyness was when he got v defensive about the Charlie Hebdo stuff, even tho he had pals patiently explaining"',
       'Reminder: Samsung Unveils the Galaxy Note 5 and S6 Edge+ Tomorrow Morning! - - #tech #news...',
       '"Gonna do a movie marathon with my friend today and tomorrow. Twilight, New Moon, Eclipse, BD1, The Runaways, Adventureland, (c)"',
       'Oscars Well makes his debut over fences at Punchestown tomorrow. It looks as though he\\u2019ll have Plan A and Darwins Fox to overcome.',
       'Q2 10:30. Philip Rivers pass to Eddie Royal to the San Diego 19 for 7 yards. 3rd & 1.',
       '@user Work colleague of mine on Thursday night: ""oh, look, they\'re showing the Bee Gees on video."" 23-yr old colleague: ""who?"""',
       "Who's going to get them Gucci foams tomorrow",
       'Yes glass of red\\u002c Rammstein and day off tomorrow (thank you @user just what I needed.',
       'Vampettes we are back on the 21st place keep tweeting with the has

In [15]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

tweet_words = list(sent_to_words(tweets[0]))

tweet_words[21][:10]

['reminder',
 'samsung',
 'unveils',
 'the',
 'galaxy',
 'note',
 'and',
 'edge',
 'tomorrow',
 'morning']

In [17]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

tweet_words = remove_stopwords(tweet_words)

def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    trigram = gensim.models.Phrases(bigram[words], min_count = tri_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod, trigram_mod

bigram_tr, trigram_tr = bigrams(tweet_words)

print(trigram_tr[bigram_tr[tweet_words[0]]][:200])

['qt', 'user', 'original', 'draft', 'th', 'book', 'remus', 'lupin', 'survived', 'battle', 'hogwarts']


In [19]:
nlp = spacy.load('en', disable=['parser', 'ner'])

warnings.filterwarnings("ignore")

def lemmatization(texts, allowed_postags=['NOUN','ADJ','VERB','ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

trigrams_tr = [trigram_tr[bigram_tr[review]] for review in tweet_words]

lemma_lg = lemmatization(trigrams_tr)

In [20]:
lemma_lg[0][:20]

id2word_lg = gensim.corpora.Dictionary(lemma_lg)
id2word_lg.filter_extremes(no_below=10, no_above=0.35)
id2word_lg.compactify()
id2word_lg.save('train_dict_lg')
corpus_lg = [id2word_lg.doc2bow(text) for text in lemma_lg]

corpus_lg[22][:10]


[(42, 1), (57, 1), (97, 1), (98, 1), (99, 1), (100, 1)]

In [27]:
from gensim.models import HdpModel
hdp = HdpModel(corpus_lg, id2word_lg, chunksize=10000)

len(hdp.print_topics())

20

In [28]:
hdp.print_topics(num_topics=20)

[(0,
  '0.003*gandhi + 0.003*sooner + 0.002*tragedy + 0.002*solution + 0.002*twitter + 0.002*stop + 0.002*occur + 0.002*cap + 0.002*jealous + 0.002*space'),
 (1,
  '0.003*captain + 0.003*gap + 0.002*spark + 0.002*surrender + 0.002*starter + 0.002*note + 0.002*shot + 0.002*going to + 0.002*appointment + 0.002*sweet'),
 (2,
  '0.003*success + 0.002*programme + 0.002*cheap + 0.002*apply + 0.002*one + 0.002*negative + 0.002*everywhere + 0.002*draft + 0.002*terrorist + 0.002*fantasy'),
 (3,
  '0.004*remove + 0.003*twitt + 0.002*lil + 0.002*cowboy + 0.002*window + 0.002*talented + 0.002*seriously + 0.002*stack + 0.002*tea + 0.002*card'),
 (4,
  '0.003*monday_morning + 0.003*release + 0.002*talent + 0.002*pay + 0.002*internet + 0.002*match + 0.002*repeat + 0.002*difference + 0.002*recall + 0.002*tease'),
 (5,
  '0.003*lie + 0.003*surrender + 0.003*performance + 0.003*social + 0.003*hopefully + 0.002*teen + 0.002*bubble + 0.002*happy + 0.002*progress + 0.002*joy'),
 (6,
  '0.003*actor + 0.003*